In [32]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 21 10:51:23 2023

@author: souchaud
"""

import os
import glob
import imageio
import numpy as np
import pandas as pd
import trackpy as tp
from tqdm import tqdm
# import functions_track_and_analyze as lib
import matplotlib.pyplot as plt
import cv2
from skimage import util
# from joblib import Parallel, delayed

In [33]:
# Consolidated parameters
PARAMS = {
        'diameter': 9,
        'minmass': 50,
        'max_size': 25,
        'separation': 15,
        'noise_size': 7,
        'smoothing_size': None,
        'invert': False,
        'percentile': 10,
        'topn': None,
        'preprocess': True,
        'max_iterations': 25,
        'filter_before': None,
        'filter_after': None,
        'characterize': True,
        'engine': 'auto',
        'threshold': 40,  # 90,
        'min_frames': 200,
        'max_displacement': 15,
        'frame_interval': 15,
        'pixel_size': 1.2773,
        'remove_exts': ['.jpg', '.svg', 'hdf5', '.png'],
        'long_time': False,
        'max_frame': 340,
        # 'data_dir': '/Users/souchaud/Desktop/A_Analyser/CytoOne_HL5/',
        'data_dir': '/Volumes/Labo_Alex_Mac/A_analyser/CytoOne_HL5/',
        # 'data_dir': '/Users/souchaud/Desktop/A_Analyser/NonT_SorC/',
        # 'output_dir': '/Users/souchaud/Desktop/Analyses/CytoOne_HL5_longtime/'
        # 'data_dir': '/Volumes/Labo_Alex_Mac/A_analyser/CytoOne_HL5/',´
        'output_dir': '/Users/souchaud/Desktop/Analyses/CytoOne_HL5_longtime_new_param/'
        # 'output_dir': '/Users/souchaud/Desktop/Analyses/NonT_SorC_longtime_New/'
        }

In [34]:
EXPERIMENT_NAMES = [f + '/mosaic/' for f in os.listdir(PARAMS['data_dir'])
                    if os.path.isdir(os.path.join(PARAMS['data_dir'], f))]

In [35]:
def compute_mean_speed(filtered):
    """
    Compute mean speed.

    Parameters
    ----------
    - filtered: DataFrame with tracked cells
    Returns
    - mean_speed: Mean speed of all particles
    - mean_speed_part: Mean speed per particle
    """
    dx = filtered.groupby('particle')['x'].diff()
    dy = filtered.groupby('particle')['y'].diff()
    displacement = np.sqrt(dx**2 + dy**2)
    duration = filtered.groupby('particle')['frame'].diff() * PARAMS['frame_interval']
    mean_speed = (displacement.sum() / duration.sum()) * PARAMS['pixel_size'] * 60
    instant_speed = displacement / duration
    mean_speed_part = instant_speed.groupby(filtered['particle']).mean() * PARAMS['pixel_size'] * 60
    return mean_speed, mean_speed_part


def clean_directory(dir_path):
    """Remove all files with the specified extensions in the directory."""
    for file in os.listdir(dir_path):
        if file.endswith(tuple(PARAMS['remove_exts'])):
            os.remove(os.path.join(dir_path, file))

In [37]:
def main():
    """Process all experiments."""
    for exp_name in EXPERIMENT_NAMES:
        print(exp_name)
        process_experiment(exp_name, PARAMS)


if __name__ == '__main__':
    main()

Frame 123: 614 features


processing batches:   0%|          | 0/3 [04:53<?, ?it/s]


KeyboardInterrupt: 